In [73]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import os
import json
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextContainer
from json import JSONEncoder
from collections import namedtuple



## Extracting from JSON from DNB Eiendom Salgsoppgaver

we need to find:<br>

price in nok - convert to int strings that start with "kr" DONE <br>
location adress and kommune DONE <br>
buldingyear- find any 4 letter string that can be converted to int and that is below 2030 DONE <br> 
energy label - color and type DONE<br>
number of rooms DONE <br>
area DONE <br>
heating



In [74]:
class Bolig:
        prisantydning = []
        byggeaar = []
        bruksareal = []
        number_of_rooms = []
        heating = []
        location = []
        energy_label = []
        property_type_description = []

In [3]:
class BoligEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

def customBoligDecoder(boligDict):
    return namedtuple('X', boligDict.keys())(*boligDict.values())

In [3]:
def checkstr(s):
        try:
            out = int(s)
            return True
        except:
            return False
        
def findPrice(boligelement):
    prices = [] 
    for el in boligelement:
            pris = [x.replace("kr","") for x in el if str.startswith(x, "kr ")]
            pris = [x.replace(" ","") for x in pris ]
            pris = [int(x) for x in pris if checkstr(x) == True]
            if pris != []:
                prices.append(pris)    
    return prices

    
    
def findArea(boligelement):
    Areas = [] 
    for el in boligelement:
            if "bruksareal" in el:
                possbileArea = el[el.index("bruksareal")+1]
                if str.endswith(possbileArea,"m²"):
                    possbileArea = possbileArea.replace(" m²","")
                    if checkstr(possbileArea) == True:
                        Areas.append(int(possbileArea))
    return list(set(Areas))

def findBuildingYear(boligelement):
    buildingYears = [] 
    for el in boligelement:
        if "byggeår" in el:
            possbileYear = el[el.index("byggeår")+1]
            if len(possbileYear) == 4 and checkstr(possbileYear) == True:
                possbileYear = int(possbileYear)
                if possbileYear > 1800 and possbileYear <= 2030:
                    buildingYears.append(possbileYear)
    return buildingYears

def findBuildingAddresse(boligelement):
    location = {"adresse":[],"kommune":[]}
    for el in boligelement:
        if "adresse" in el:
            location["adresse"].append(el[el.index("adresse")+1])
        if "kommune" in el:
            location["kommune"].append(el[el.index("kommune")+1])
    ## fix formatting
    if location["adresse"] != []:
        location["adresse"] = [str.strip(x) for x in location["adresse"] if x != '']
        location["adresse"] = list(set(location["adresse"]))
        if len(location["adresse"]) > 1:
            location["adresse"] = location["adresse"][-1]
        elif len(location["adresse"]) == 1:
            location["adresse"] = location["adresse"][0]
    if location["kommune"] != []:
        location["kommune"] = location["kommune"][-1]
    answer = []
    answer.append(location["adresse"])
    answer.append(location["kommune"])
    if answer[0] == [] or answer[1] == []:
        return None
    return ", ".join(answer)


def findEnergyLabel(boligelement):
    energyLabel = [] 
    for el in boligelement:
            if "energikarakter" in el:
                karakter = el[el.index("energikarakter")+1]
                if  str.strip(karakter) != "ikke angitt" and str.strip(karakter) != '':
                    energyLabel.append(karakter)
            if "energifarge" in el:
                farge = el[el.index("energifarge")+1]
                if str.strip(farge) != "ikke angitt" and str.strip(farge) != '':
                    energyLabel.append(farge)
    if energyLabel == []:
        return None
    return energyLabel


def findNumberOfRooms(boligelement):
    numberOfRooms = [] 
    for el in boligelement:
            if "antall rom" in el:
                nRooms = el[el.index("antall rom")+1]
                if nRooms != '' and checkstr(nRooms) == True:
                    numberOfRooms.append(int(nRooms))
    if numberOfRooms == []:
        return None
    return numberOfRooms
               
def Mapper(fileJson):
    bolig = Bolig()
    bolig.prisantydning = fileJson["prisantydning"]
    bolig.byggeaar = fileJson["byggeaar"]
    bolig.bruksareal = fileJson["bruksareal"]
    bolig.number_of_rooms = fileJson["number_of_rooms"]
    bolig.heating = fileJson["heating"]
    bolig.location = fileJson["location"]
    bolig.energy_label = fileJson["energy_label"]
    bolig.property_type_description = fileJson["property_type_description"]
    return bolig
    
    
            

In [10]:
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
counterPrices = 0
counterWithoutPrices = 0
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        ## how many empty string there are in each property
        ##find price
        prices = []
        thisprice = findPrice(boligelement=bolig.prisantydning)
        if thisprice != None:
            prices.extend(thisprice)
        
        thisprice = findPrice(boligelement=bolig.byggeaar)
        if thisprice != None:
            prices.extend(thisprice)
        
        thisprice = findPrice(boligelement=bolig.bruksareal)
        if thisprice != None:
            prices.extend(thisprice)
        
        thisprice = findPrice(boligelement=bolig.number_of_rooms)
        if thisprice != None:
            prices.extend(thisprice)
        
        thisprice = findPrice(boligelement=bolig.heating)
        if thisprice != None:
            prices.extend(thisprice)
        
        thisprice = findPrice(boligelement=bolig.location)
        if thisprice != None:
            prices.extend(thisprice)
        thisprice = findPrice(boligelement=bolig.energy_label)
        if thisprice != None:
            prices.extend(thisprice)
        if prices != []:
            #print(max(prices))
            counterPrices +=1
        else:
            #print("could not find price")
            counterWithoutPrices +=1
            
        i+=1
print(i)
print(f"with prices {counterPrices}")
print(f"without prices {counterWithoutPrices}")


686
with prices 573
without prices 112


In [23]:
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
withAreas = 0
wihtoutAreas = 0
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        ## how many empty string there are in each property
        ##find price
        Areas = []
        thisarea = findArea(boligelement=bolig.prisantydning)
        if thisarea != []:
            Areas.extend(thisarea)
        
        thisarea = findArea(boligelement=bolig.byggeaar)
        if thisarea != []:
            Areas.extend(thisarea)
        
        thisarea = findArea(boligelement=bolig.bruksareal)
        if thisarea != []:
            Areas.extend(thisarea)
        
        thisarea = findArea(boligelement=bolig.number_of_rooms)
        if thisarea != []:
            Areas.extend(thisarea)
        
        thisarea = findArea(boligelement=bolig.heating)
        if thisarea != []:
            Areas.extend(thisarea)
        
        thisarea = findArea(boligelement=bolig.location)
        if thisarea != []:
            Areas.extend(thisarea)
            
        thisarea = findArea(boligelement=bolig.energy_label)
        if thisarea != []:
            Areas.extend(thisarea)
            
        Areas = list(set(Areas))
        if Areas != []:
            withAreas += 1
        else:
            wihtoutAreas +=1
        i+=1
print(i)
print(f"with areas {withAreas}")
print(f"without areas {wihtoutAreas}")


686
with areas 485
without areas 200


In [20]:
## Building Year
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
withValues = 0
wihtoutValues = 0
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        ## how many empty string there are in each property
        ##find price
        Values = []
        thisValue = findBuildingYear(boligelement=bolig.prisantydning)
        if thisValue != []:
            Values.extend(thisarea)
        
        thisValue = findBuildingYear(boligelement=bolig.byggeaar)
        if thisValue != []:
            Values.extend(thisValue)
        
        thisValue = findBuildingYear(boligelement=bolig.bruksareal)
        if thisValue != []:
            Values.extend(thisValue)
        
        thisValue = findBuildingYear(boligelement=bolig.number_of_rooms)
        if thisValue != []:
            Values.extend(thisValue)
        
        thisValue = findBuildingYear(boligelement=bolig.heating)
        if thisValue != []:
            Values.extend(thisValue)
        
        thisValue = findBuildingYear(boligelement=bolig.location)
        if thisValue != []:
            Values.extend(thisValue)
            
        thisValue = findBuildingYear(boligelement=bolig.energy_label)
        if thisValue != []:
            Values.extend(thisValue)
        Values = list(set(Values))
        ##print(bolig.location)
        if Values != []:
            withValues += 1
        else:
            wihtoutValues += 1
        i+=1
print(i)
print(f"with buildingyear {withValues}")
print(f"without buildingyear {wihtoutValues}")


686
with buildingyear 564
without buildingyear 121


In [68]:
## adresse
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
withValues = 0
wihtoutValues = 0
RepeatingWords = {}
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        Value = []
        thisValue = findBuildingAddresse(boligelement=bolig.prisantydning)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        thisValue = findBuildingAddresse(boligelement=bolig.byggeaar)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findBuildingAddresse(boligelement=bolig.bruksareal)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findBuildingAddresse(boligelement=bolig.number_of_rooms)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findBuildingAddresse(boligelement=bolig.heating)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findBuildingAddresse(boligelement=bolig.location)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
            
        thisValue = findBuildingAddresse(boligelement=bolig.energy_label)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        i+=1
        if Value != []:
            withValues +=1
        else:
            wihtoutValues +=1
print(i)
print(f" buildings with adress {withValues}")
print(f" buildings without adress {wihtoutValues}")
        


686
 buildings with adress 574
 buildings without adress 111


In [80]:
## energyLabel
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
withValues = 0
wihtoutValues = 0
RepeatingWords = {}
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        Value = []
        thisValue = findEnergyLabel(boligelement=bolig.prisantydning)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        thisValue = findEnergyLabel(boligelement=bolig.byggeaar)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findEnergyLabel(boligelement=bolig.bruksareal)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findEnergyLabel(boligelement=bolig.number_of_rooms)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findEnergyLabel(boligelement=bolig.heating)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        
        thisValue = findEnergyLabel(boligelement=bolig.location)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
            
        thisValue = findEnergyLabel(boligelement=bolig.energy_label)
        if thisValue != None and thisValue not in Value:
            Value.append(thisValue)
        i+=1
        if Value != []:
            withValues +=1
        else:
            wihtoutValues +=1
print(i)
print(f" buildings with energyLabel {withValues}")
print(f" buildings without energyLabel {wihtoutValues}")
        


686
 buildings with energyLabel 409
 buildings without energyLabel 276


In [97]:
## findNumberOfRooms
i = 1
folderUrl = "./DNB/JSON/Untreated/Leilighet/"
withValues = 0
wihtoutValues = 0
RepeatingWords = {}
for file in os.listdir(folderUrl):
    #print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        Value = []
        thisValue = findNumberOfRooms(boligelement=bolig.prisantydning)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        thisValue = findNumberOfRooms(boligelement=bolig.byggeaar)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        
        thisValue = findNumberOfRooms(boligelement=bolig.bruksareal)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        
        thisValue = findNumberOfRooms(boligelement=bolig.number_of_rooms)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        
        thisValue = findNumberOfRooms(boligelement=bolig.heating)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        
        thisValue = findNumberOfRooms(boligelement=bolig.location)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
            
        thisValue = findNumberOfRooms(boligelement=bolig.energy_label)
        if thisValue != None and thisValue not in Value:
            Value.extend(thisValue)
        i+=1
        Value = list(set(Value))
        if Value != []:
            withValues +=1
        else:
            wihtoutValues +=1
print(i)
print(f" buildings with room numbers {withValues}")
print(f" buildings without room numbers {wihtoutValues}")

686
 buildings with room numbers 500
 buildings without room numbers 185


In [110]:
garbage_list = ["oppvarming", "leiligheter", "leilighet", "leilighetene", "oppvarmes"
                , "fra", "med", "forbruk", "av", "varmt", "tappevann", "og", "varme"
                , "måles", "", "pr", "leilighet", "kostnaden", "kommer", "i", "tillegg"
                , "til", "sameiets", "stipulerte", "fellesutgifter", "leveres", "alle"
                , "rom", "unntatt", "bod", "soverom", "det", "blir", "gulvene,", "unntak"
                , "badet", "som", "kan", "bli", "levert", "tilvalgsproses-", "sen", "kan"
                , "det", "velges",  "flere", "rom", "det", "monteres", "målere", "hver"
                , "som", "måler", "forbruket", "a-konto", "er", "ikke", "inkludert"
                , "endelig", "budsjett", "fastsettes", "på", "årsmøte", "tas", "forbehold"
                , "om", "endringer", "felles-", "kostnader,", "da", "dette", "er", "basert"
                , "på", "erfarings-", "tall", "fordelingsnøkkel", "for", "fordeling", "kostnader"
                , "reguleres", "vedtekter,", "bygger", "utgangspunktet", "på", "sameier-", "brøk"
                , "enkelte", "elementer", "budsjettet""skjer", "ved", "bad", "utstyrt", "vil", "ha"
                , "egne", "separate", "energimålere", "energi", "produksjon", "varmtvann", "enten"
                , "lokalt", "eller", "sentralt", "varmen", "ringeanlegg", "energi", "produksjon", "varmtvann"
                , "enten", "lokalt", "eller", "sentralt", "varmen", 'budsjettet', "skjer"
                , "stue","gulv", "andel", "beregnet", "normalisert", "klima:", "10238", "kwh/m²"
                , "sum", "andel", "netto", "oppvarmingsbehov:", "1000", "%","tilkoblet", "via", "rør-i-rør"
                , "system", "samlestokk", "skap/sjakt", "stoppekraner", "rørskap", "(må", "betjenes", "vertøy)"
                , "opplysninger", "borettslaget:", "skifting", "vannrør", "bunnledninger", "utført", "perioden"
                , "1999-2005", "mekanisk", "ventilasjon", "avtrekk", "kjøkken", "tilluft", "boligen", "via"
                , "veggventiler/vindusventiler", "felles", "tilstandsgrad", "satt", "ut", "funksjon","tilkoblet"
                , "via", "sikringsskap", "automatsikringer", "strømmåler", "plassert", "fellesgang", "rett"
                , "utenfor", "leiligheten", "20", "a", "hovedsikring", "begrenset", "kapasitet"
                , "røykvarslere/brannvarslere", "brannslukkingsapparat", "tilkoblet", "via",'"skifting', '1999-2005"'
                ,"samt", "fyring", "(a-konto)", "husleien", "soverommene", "bad,", "wc/dusj", "wc", "økonomiske"
                , "leiligheten:", "felleskostnader:", "totale", "felleskostnader", "dd", "kr", "8822,–"
                , "md", "herav:", "garasje", "felleskostnader", "akonto", "kapitalkostnader", "obos01"
                ,"oppvarming/varmtvann:", "stue/", "flislagt", "baderom", "innredet", "servantinnredning,"
                , "dusjkabinett,", "vegghengt", "toalett", "opplegg", "vaskemaskin", "stue/kjøkken"
                ,"varme,", "tilknyttet","og/", "gjøres", "oppmerksom", "at", "beboelsesrom", "mangle"
                , "oppvarmingskilde,", "kjøper", "overtar", "slik", "den", "fremstår", "visning", "teknisk"
                , "installasjon", "oppvarming:","forbruksvann", "felleskostnadene", "standard", "beskrivelse;"
                , "-", "primæroppvarming", "vegg", "resten", "-", "balansert", "styres", "smart", "-", "fiber"
                , "tv/", "internett", "-", "porttelefon", "-", "kombi", "kjøl/", "frys,", "komfyr,", "platetopp"
                , "oppvaskmaskin", "-", "flis", "gulv,", "dusj,", "servant", "skuffeseksjon", "vaskemaskin/", "tørk"
                , "-", "fullsprinklet", "utvendig", "sprinklet", "balkonger", "inntrukket", "bygget", "-", "smart"
                , "lock", "vannavstegning", "hoveddør", "-", "har", "belysning", "led", "lampe", "tak", "resten"
                , "har", "vært", "bebodd,", "derav", "ingen", "estimat", "strømforbruk",  "boligen/leiligheten"
                , "varmekilder", "funksjonstestet", "takstmann", "foretatt", "kontroll", "hvorvidt", "en"
                , "varmekilde", "tg", "vurdert","dersom", "fastmonterte", "så", "følger", "heller", "disse"
                , "rommene", "avviker", "bransjenormen,", "men", "tilpasset", "klimaet", "området","installeres"
                , "varmeanlegg", "leiligheter,", "form", "hovedbad", "montert", "gang", "mot", "entrè"
                , "overtagelsen,", "svarer", "boligens", "utgifter,", "herunder", "borettslaget,", "strøm,", "mv"
                , "antatt", "kostnad", "knyttet", "estimert", "9kr/m2", "mnd", "(akonto),", "installasjon,", "måling"
                , "avregning", "avregning", "10-årig", "leieavtale", "trådløs", "fjernavlesning", "målerverdier", "vann"
                , "ønskes", "avtalen", "oppsagt", "tidligere", "må", "borettslaget", "betale", "gjenværende"
                , "investeringskostnaden", "7-1", "(1)", "eiendommen", "gjelder", "bolig", "skal", "fordeles", "mellom"
                , "andelseierne", "fremgår", "bygge-", "finansieringsplanen", "betales", "avregnes", "gang", "år"
                ,"2", "gitt", "grunn", "alder", "forhold", "forventet", "levetid", "varmes", "opp", "følge", "eier"
                , "lagt", "følgende", "rom:","øvrig", "elektrisitet", "energimerking", "selger", "ansvarlig"
                , "energiattest", "frittstående", "bygninger", "under", "50", "kvm", "foreligger", "energiattest"
                , "denne", "oppvarmingskarakter", "rød", "‑", "energikarakter", "f","ligger", "åpen", "løsning,",
                "stuen", "utgang", "østvendt,", "overbygget", "balkong", "ca", "7", "stuen", "plass", "sofagruppe",
                "spisebord", "foran", "vinduet", "kjøkkenet", "innredning", "ikea", "her", "over-", "underskap", "glatte", "fronter,",
                "laminat", "benkeplate", "hvitevarer", "et", "eldre", "oppussingsbehov", "stråleovn", "vaskerom", "borettslagets",
                "kjeller", "teknisk:", "kobber", "plast", "avløpsrør", "plast", "avløpsrør", "soil", "naturlig", "120", "liters",
                "varmtvannstank", "kjøkkeninnredning", "trappegang", "hovedsikring/", "overbelastningsvern:", "25a", "skjult",
                "anlegg", "registrert", "brannvarsler", "brannslukningsapparat", "registrert","begge", "badene", "oppholdsrom",
                "innvendig",  "etter", "eierbrøk", "inkluderer", "(a‑konto)", "se", "inkluderer", "bla", "styrehonorar,",
                "forretningsførerhonorar,", "konsulenthonorar,", "vaktmesterhonorar,", "drift‑", "vedlikehold,", "kabel‑tv/",
                "internett,", "forsikring,", "noen", "kommunale", "avgifter,", "oppvarming,", "renhold,", "mm", "fakturert",
                "-sikringsskap", "entré", "1etg", "2etg", "kjøkken," ,"bad/", "baderommene", "moderne", "hev/", "senk", "glass", 
                "nordpeis", "synlige", "stoppekran", "ny", "viftemotor", "loft", "2021", "lufteventiler", "vinduer", "strøm",
                "mulighet", "montering", "gang,", "utstyr", "testet", "høiax", "varmtvannsbereder", "liter", "byggeår,", 
                "kjøkkenbenk", "sikringer", "består", "totalt", "15", "kurser", "henhold", "kursfortegnelse,", "50a",
                "røykvarsler", "brannslange", "kjøkkenbenk", "tekniske", "installasjoner:", "vannledninger:", "vannledninger"
                , "plastrør", "(rør", "rør)", "avløpsrør:", "synlig", "ventilasjon:", "vannbåren", "fordelerskap"
                , "varmtvannstank:", "varmtvannsberedere", "brannslukking:", "sprinkleranlegg", "anlegg:", "sikringsskapet"
                ,"fordelerskap", "himling", "leilighetenes", "røranlegg", "(varmt-", "kaldtvann)", "fordelerskapene", "relativ"
                , "store", "plassering", "besluttet,", "være", "del", "detaljprosjekteringen", "fordeler-", "skapene", "individuelt"
                , "kaldt", "vann,", "faktureres", "direkte", "utenom", "ventilasjon:", "detaljering", "varmesystem:", "elementpipe,"
                , "installert","gulvene", "tilvalgsproses", "tv", "kjøl", "frys", "komfyr", "dusj", "bransjenormen", "løsning",
                "østvendt", "over", "fronter", "a‑konto", "oppvarmet", "varmtvannssystem", "hev",  "byggeår", "kursfortegnelse",
                "fliser", "øvrige", "rør", "kaldtvann", "besluttet", "fordeler", "elementpipe", "eksklusive", "egen", "energimåler", 
                "registrerer", "energiforbruket", "smarte", "termostater", "regulering", "konto", "faktisk", "boligene", "tilknyttes",
                "eidsiva", "hovedenergikilde", "målt", "fellesareal", "går", "tilførselsrør", "gulvet", "tilfra", "de", "ulike",
                "gulvsonene", "stedet", "vinduer", "sett", "lys", "normal", "bruksslitasje", "ytterdører", "dørhåndtak", "morløst",
                "bør", "justeres", "slitasje", "innvendige", "dører", "overflater", "gulv", "pågrunn", "generelt", "slitte", "merker",
                "riper", "sprekker", "gulvoverflater", "terrasser", "ol", "kun", "hovedkonstruksjon", "ansvarsområde", "rekkverkshøyden",
                "fraviker", "gjeldende", "forskrifter", "rekkverk", "", "m", "ledningsnett", "sanitærinstallasjoner", "vanntrykk", "avløp"
                , "undersøkt", "funnet", "ok", "tilstandsvurdering", "enkle", "observasjoner", "undertegnede", "spisskompetanse", "sintef"
                , "byggforsk", "bruk", "anbefalt", "brukstid", "", "avhengig", "type", "", "påvirkende", "faktorer", "røropplegget"
                , "tilfellet", "oversteget", "mer", "enn", "halvparten", "derfor", "generelt", "vurdering", "mangler", "luftespalte"
                , "dør", "medfører", "sirkulasjon", "anbefales", "å", "etablere", "friskluftventiler", "tilstrekkelig", "lufting", "grunnlag"
                , "etableres", "avtrekksvifte", "våtrom", "begrense", "fuktpåkjenning", "rommet", "–", "bad", "noe", "svakt", "fall", "sluk"
                , "vanntett", "sjikt", "membran", "elektronisk", "ventilasjonsanlegg", "badegulvet", "hele", "hvor", "skrusikringer", "stue"
                , "malt", "platekledning", "tapet", "vegger", "ekstra", "kalde", "dager", "aggregat", "frem", "ventilasjons", "entre", "luft"
                , "", "calling", "tale", "kamerafunksjon", "ahs", "div", "jordfeilsautomater", "vegg‑", "handelen", "roms", "underetasje"
                , "markerrasse", "gateparkering", "beliggende", "lysaker", "fornebu", "gangavstand", "buss", "korte", "avstander", "barnehage"
                , "skoler", "forretninger", "el", "varmtvanns", "bereder", "ventilator", "kullfilter", "ventiler", "røykvarslere", "avdelt"
                , "godt", "både", "sofa", "spisegruppe", "oppvarmingskilde", "overtas", "tilknyt", "tet", "elsentral", "plasseres", "tekniskrom"
                , "fellesvarmtvannsberedere", "teknis", "krom", "oppleggskap", "gassflasker", "ter", "rassebalkong", "tilvalg", "forøvrig"
                , "tilgjengelighetskrav", "lysforhold", "•", "påtar", "seg", "hms", "helse", "miljø", "sikkerhet", "ansvar", "byggherreforskriften"
                , "terrenglinjer", "fasadetegning", "illustrerende", "bindende", "samsvar", "lydkravene", "ns", "", "klasse", "c", "lydklassen"
                , "tilfredsstiller", "kravene", "ellers", "baderommet", "diverse", "ekstraordinær", "generalforsamling", "", "styret", "foreslår"
                , "gå", "bort", "dagens", "varmtvannskostnader", "tar", "utgangspunkt", "gjennom", "drift", "sameiebrøken", "sameiet", "ser", "nødt"
                , "øke", "posten", "gjennomsnittsum", "", "", "variere", "sameiebrøk", "avvik", "", "kostnadsføres", "regnskap", "", "murt"
                , "teglsteinspipe", "tilsluttet", "ildsted", "overflater", "vegger", "observert", "bruksmerker", "plasser", "himling", "rissdannelser"
                , "taket", "tildeles", "lignende", "håndløper", "råtten", "fornyes", "varmtvann", "", "kobberrør", "", "", "støpejern", "hovedstoppekran"
                , "dato", "risiko", "funksjonstesting", "tilstandsvurderingen", "ble", "opprinnelig", "oppdriftsventilasjon", "lenger", "ventil", "gi"
                , "redusert", "luftsirkulasjon", "referansenivå", "luftbehandling", "fuktindikator", "benyttet", "påvist", "indikasjoner", "fukt", "utover"
                , "vanlige", "verdier", "våte", "soner", "typen", "konstruksjon", "normalt", "", "konstruksjonen", "oppnådd", "anvendt", "veggoverflater"
                , "våtsone", "fast", "inventar", "sanitærinstallasjoner", "wc", "defekte", "a’", "grunnpakke", "tvinternett", "avgifter", "renovasjon"
                , "fellesarealer", "vedlikehold", "garasjeanlegg", "bygg", "vvs", "elektro", "heis", "forretningsførsel", "revisjon", "forsikring"
                , "sikringsfonder", "reduserte", "strømkostnader", "dobbeltmandlet", "", "systemet", "stipulert", "senere", "avregnet", "d", "god"
                , "planløsning", "trivelig", "ei", "romslig", "sofakrok", "fin", "flott", "", "integrerte", "skyvedørsgarderobe", "uteområdet", "lunt"
                , "markterrasse", "hage", "svært", "gode", "solforhold", "utebod", "lagring", "eplestien", "vi", "styring", "futurehome", "du", "oppnår"
                , "energieffektiv", "gir", "lavere", "utgifter", "bra", "deg", "miljøet!", "etablert", "røropplegg", "besiktiget", "filter", "byttet", "siden"
                , "", "", "a", "pulverapparat", "forblendet", "pipeløp", "fremtidig", "tilkobling", "termostat", "installasjonene", "takstmannens"
                , "kompetansefelt", "ytterligere", "gjennomgang", "eltakst", "opplyser", "nye", "gass", "deler", "ventilasjonssytem", "sanitær", "nødvendig"
                , "va", "brannslukkeanlegg", "sanitæranlegg", "boligformål", "lave", "strømutgifter", "medfølger", "avtale", "", "amp", "hovedsikringer", ""
                , "anlegget", "jordfeilbryter", "brannvarslingsanlegg", "sprinkelanlegg", "langs", "vinduene", "ned", "blitt", "legges", "båren", "kabler"
                , "badegulv", "utføres", "fellesmåler", "fellesmåleren", "vannforbruk", "internt", "fastmontert", "salget", "info", "energiklasse"
                , "energimerket", "grønn", "fellesutgiftene", "", "", "brannvarslere", "ett", "brannslokningsapparat", "byggemåte", "enkel"
                , "bygningsbeskrivelse", "lavblokk", "", "etasjer", "primærareal", "m", "plan", "", "flatt", "trekonstruksjon", "tekket", "folie"
                , "papp", "yttervegger", "tre", "betongkonstruksjoner", "forblending", "fasade", "teglstein", "trevinduer", "lags", "isolerglass"
                , "utvendige", "persienner", "ytterdør", "treverk", "leilighetsdør", "brann", "lydmotstand", "kikkhull", "intercom", "åpner", "elanlegg"
                , "hovedsak", "terrasse", "vest", "garasjeplass", "parkeringskjeller", "vaskerommet", "varmtvannsforsyning", "forsyner", "regulator"
                , "fordelerstamme", "stedvis", "vegghengte", "bederom", "også", "forniklet", "rørirør", "roth", "multipex", "lokalisert"
                , "overløp", "oppgradert", "ifm", "våtromsrehabilitering", "", "hovedstrekk", "friskluftstilførsel", "veggventiler", "luftespalter"
                , "varmesentral", "varme", "andre", "downlightsbelysning", "dørtelefon", "telia", "tilknytning", "kontrolleres", "årlig", "regi", "stuer"
                , "vaskerom", "aktuell", "byggeforskrift", "dokumentasjon", "", "dokumentasjon", "malte", "folierte", "himlingsplater", "plastsluk"
                , "smøremembran", "ukjent", "utførelse", "nærmere", "inspisert", "dusjkabinett", "lite", "servantskap", "nedfelt", "gulvstående"
                , "kjøkkenventilator", "hulltaking", "fysisk", "mulig", "pga", "tilliggende", "konstruksjoner", "bortsett", "bodrom", "mursteinspipe"
                , "ibruk", "idag", "opplyst", "var", "sotluke", "videre", "innstallering", "tilsyn", "godkjenning", "feier", "tilrettelegges", "installere"
                , "badwc", "romskjema", "gjestetoalettwc", "uten", "oppvamring", "lufttilførsel", "kvaliteter", "oppgraderinger", "roms", "selveierleilighet"
                , "", "parkering", "carport", "trappefri", "adkomst", "praktisk", "sydøstvent", "", "pusset", "", "lagrinsplass", "avtrekksanlegg", "", "l", ""
                , "hafslund", "individuell", "hvert", "gulvføler", "hjørneleilighet", "", "etasje", "heisadkomst", "sosial", "kjøkkenløsning", "to", "solrike"
                , "henholdsvis", "m²", "m²", "vendt", "nordvest", "høyt", "fritt", "usjenert", "utsikt", "akto", "inkl", "fk", "", "behov", "modernisering"
                , "man", "vente", "delvis", "adskilt", "spiseplass", "vindusflater", "lagringsplass", "tilhørende", "velkommen!", "stuemedfølger", "original"
                , "bevart", "ovninnsats", "glassfront", "forbrenning", "arealeffektiv", "‑roms", "populært", "område", "inkvv", "oppvarminggarasjeplassmå"
                , "oppleves!", "tid", "byggets", "gulvstøp", "", "tilkomst", "gangen", "stand", "hjelp", "tilrettelagt", "stikkontakter", "fordelt", "rundt"
                , "utvidet", "innebygde", "jordfeilbrytere", "fordelingskurser", "kjellerbod", "når", "sist", "oppdatert?", "elarbeid", "ifbm", "installering"
                , "nytt", "bb", "bjørn", "bakke", "as", "kjenner", "første", "omgang", "oppdatert", "stemmer", "antall", "elfagmann", "drifts"
                , "vedlikeholdskostnader", "fellesgjeld", "avhenge", "rente", "avdragsvilkår", "felleslånet", "hvilke", "tjenester", "ønsker", "kostnadene"
                , "bestemmelser", "vedtektene", "månedlige", "renter", "ev", "avdrag", "driftsår", "prislisten", "bygningsmassen", "premie"
                , "husleiegarantiforsikring", "vaktmestertjenester", "snømåking", "renhold", "driftskostnader", "", "sistnevnte", "anse", "akontobeløp"
                , "kjøpers", "faktiske", "innlagt", "veksler", "vist", "toalettrom", "pipegjennomføring", "medfølge", "romfølere", "lille", "boden"
                , "vannet", "elkassett", "sportsbod", "beboerne", "elskap", "e", "levers", "fordi", "raskere", "responstid", "ønske"
                , "temperaturregulering", "temperatur", "oppsett", "ønskelig", "kombinert", "toppmodell", "smartløsning", "tibberintegrasjon"
                , "angir", "energistandard", "varmingskarakter", "figuren", "symboliseres", "hus", "fargen", "viser", "oppvarmings", "karakter"
                , "bokstaven", "stort", "gulvareal", "eikespiler", "stavs", "eikeparkett", "lekre", "fargetoner", "pvc", "vannskap", "fordelingsskap"
                , "baderomshimling", "forskriftsmessig", "avrenning", "baderomsgulv", "yttervegg", "manuelt", "pt", "ute", "jobber", "få", "ordnet"
                , "konferer", "megler", "oppdateringer", "‑", "betjening", "vaktmester", "bygningen", "eiendomsskatt", "honorar", "informerer", "hun"
                , "brukt","vet", "tilstanden", "måleravlesning", "tappe", "tilfør", "selsrør", "fjern", "trolig", "alternativt", "indi", "viduell"
                , "energianlegget", "fungerer", "hovedoppvarmingskilde", "tenkt", "sonestyrt", "sje", "bærekraftig", "hall", "leverer"
                , "ventilasjonsanlegget", "viftekonvektor", "vinduslufting", "gjennomlufting", "tiltak", "unngå", "overoppvarming", "parkett"
                , "panelplater", "overflate", "lysinnslipp", "rørinstallasjon", "avløpsanlegg", "varmtvannsbereder", "blokka", "tilgang", "inspeksjon"
                , "oppvarmingen", "hovedsaklig", "vestvendt", "m²", "fint", "utsyn", "blokk", "garderobeskap", "kort", "bussholdeplass", "søke"
                , "leie", "pplass", "venteliste", "installasjoner", "rørledninger", "besiktet", "åpning", "avtrekksventil", "veggventil"
                , "spalteventiler", "brannslangetrommel", "vannsystemet", "brannvarslingssystem", "missfarget", "kondens", "varmtvannsanlegg"
                , "arealsmart", "roms", "veletablert", "barnevennlig", "boligområde", "tomasjordnes", "litt", "nord", "tromsdalen", "tromsø"
                , "sentrum", "innehar", "vedlikeholdt", "tatt", "vare", "foregår", "der", "gjør", "vannutgiftene", "holdes", "får", "behagelig"
                , "lyse", "lange", "stor", "garderobeløsning", "oppbevaringsmuligheter", "blant", "annet", "uteklær", "sko", "samme", "leilighetens"
                , "soverommet", "størrelse", "seng", "komplett", "arbeid", "veggmontert", "halv", "glassvegg", "speil", "skapplass", "tørketrommel"
                , "luke", "skap", "bergvarme", "omfatter", "byggforsikring", "altibox", "ledbelysning", "inn", "automatisk", "forbruksavlesning"
                , "seksjon", "månedlig", "ákonto", "foretas", "nyere", "lekker", "leil", "mye", "overbygd", "", "delikat", "pent", "innbydende"
                , "oppbevaring", "vei", "grilstad", "rikelig", "servicetilbud", "nydelige", "ladestien", "vvann", "varmtvannstanker", "gjennomgående"
                , "‑roms", "solrik", "fantastisk", "sjønært", "lus", "luftig", "allrom", "tvmøblement", "spise", "", "sittegrupper", "møblement"
                , "snøbrøyting", "gressklipping", "", "overspenningsvern", "gulver", "‑", "forskudd", "måned", "utfyllende", "beskrivelse", "materialer"
                , "detaljer", "leveransebeskrivelse", "kvaliteter", "pen", "toppleilighet", "eget", "bakkeplan", "innfri", "andelens", "utgjøre", "‑"
                , "styre", "trappevask", "per", "renovasjonsavgift", "", "bredbånds", "seksjonmnd", "beliggenhet", "kabeltv", "bredbånd", "fellesutgiftene!"
                , "loftsetasjen", "dører", "kobles", "sentralanlegget", "benytter", "eksisterende", "varmerør", "jordvarmeanlegg", "bud", "eiendommens"
                , "faste", "løpende", "vedlikeholdsfond", "driftsutgifter", "iht", "omfatte", "gjensidige", "budsjettert", "", "mndinkludert"
                , "bindingstid", "ferdigstillelse", "utgjør", "", "overtakelse", "forsikret", "utbygger", "overdragelse", "bebyggelsen"
                , "andelshaver", "selv", "sørge", "innboforsikring", "generalforsamlingen", "erfaringstall", "forretningsfører"
                , "vedtekter", "areal", "likt", "andeler", "ovn", "befaring", "vis", ">", "avvik", "oppvarmingsmulighet", "borte"
                , "hyttene", "uno", "tilsvarende", "wifi", "styrt", "mannsboliger", "elkasett", "rekkehus", "baderom\\vaskerom"
                , "boligrom", "ventilasjonssystem", "skapet", "hovedinntak", "vannmåler", "hovedstamme", "stakeluke", "røret", "løs"
                , "hovedsakelig", "strømskapet", "trapperom", "pipe", "feiet", "g", "meget", "sentral", "attraktiv", "populære", "ensjø"
                , "utbyggergaranti", "", "opplyses", "pipa", "koble", "rehabilitering", "ferdigstilt", "våren", "iflg", "vindusventiler"
                , "muliggjør", "innsetting", "", "funksjonelt", "belegg", "våtromsplater", "servantinnredning", "overskap", "internetttv"
                , "interessenter", "oppfordres", "undersøke", "finnes", "vermekonvektor", "herlig", "", "strekker", "sidene", "muligheter"
                , "nyte", "solen", "sommeren", "oppført", "", "", "berederen", "elkasset", "hovedbryter", "desember", "betjeningssentral"
                , "står", "bebodd", "funksjonen", "oppvarmingskildene", "oso", "", "originale", "eventuell", "etablering", "fagfolk"
                , "rette", "instanser", "formelt", "forutsettes", "branntekniske", "krav", "kontrollert", "godkjent", "stedlige"
                , "redningsetaten", "havgløtt", "oslofjorden", "ønsket", "spisestue", "vindusflatene", "skaper", "romfølelse", "overflatene"
                , "tidsriktige", "farger", "stavs", "forlengelse", "varmere", "balkongen", "fremkommer", "lånekostnader", "følgende"
                , "husleiegarantiforsikringsikringsordning", "hhv", "", "", "overunder", "", "m", "", "inngår", "påløper", "oppgitt"
                , "kostand", 'a""' , "kontobeløp", "oppgitte", "påregnes", "justering", "innflyttingstidspunkt", "organisering", "borettslag"
                , "oppdeling", "byggetrinn", "påvirke", "driftsbudsjettet", "fellesgjelden", "avdragsfri", "mens", "deretter", "økning", "periode"
                , "vises", "kolonne", "hammerdalen", "berederanlegg", "forvarmes", "riktig", "elkraft", "medgått", "vask", "walkincloset", "benkeskap"
                , "", "oppvaring", "tiltak", "annen", "topp", "endeleilighet", "populær", "amalienborg", "vågsbygd", "holder", "således", "trukket"
                , "varerør", "lukke",  "jordfeilautomater", "", "", "varierende", "grad", "memekanisk", "åpent", "prospekt", "jm", "imidlertid"
                , "boliger", "prosjekteres", "minst", "%", "bedre", "tek", "romoppvarming", "vifter", "pumper", "ol", "myndighetskravet", "refererer"
                , "søknadspliktige", "knytter", "kunne", "boligblokk", "klimaskallet", "sammen", "sikrer", "lavt", "energiforbruk", "jevn"
                , "temperaturfordeling", "trekk", "moderniseringer", "påkostninger", "", "erstattet", "skiftet", "radiatorkraner", "stad"
                , "kuleventiler", "+", "plugging", "ubrukte", "radiatorkurser", "ytterdørene", "rehabilitert", "porttelefoner", "", "utbedring"
                , "trapp", "inngang", "", "", "prefabrikerte", "trapper", "oppganger", "", "brannsikre", "kjellerdører", "røykluker", "trappeløp"
                , "låssystem", "fellesdører", "oppgradering", "utebelysning", "oppussing", "næringslokaler", "", "brannbalkonger", "innkjøp", "port"
                , "bakgården", "", "brannslukkingsapparater", "fellesområder", "bytting", "gamle", "lysarmaturer", "utarbeidelse", "tilstandsrapport"
                , "vedlikeholdsplan", "", "maling", "ytterdører", "spyling", "vaskemaskiner", "fellesvaskeriet", "‑", "branndører", "", "bakgård", "‑"
                , "nøkkelinfo", "adresse", "", "arnljot", "gellines", "gnr", "", "bnr", "oslo", "kommune", "priser", "", "", "omkostninger", "", "ref"
                , "salgsoppgave", "bra", "", "prom", "soverom", "", "eierform", "boligtype", "felleskostnadeneinkluderer", "forslag", "utarbeidet"
                , "obos", "driftsåret", "", "byggene", "vedlikeholdskostnadene", "boligenes", "bradels", "kabeltvinternett", "beløp", "", "jfr"
                , "budsjettforslag", "fås", "henvendelse", "radiatoranlegg", "etc", "fastsette", "dekning", "andelenes", "bruksenhetene", "kreves"
                , "avtalt", "eiendomsskatt", "kværnerhaøyden", "belastet", "eiendomskatt", "budsjetterte", "eiendomsskattekontoret", "beslutte"
                , "viderefakture", "skatt", "fellesgjeld", "finanseres", "annuitetslån", "års", "avdragsfrihet", "lånet", "løper", "flytende"
                , "rentesatsen", "dag", "hensyntatt", "garasje", "kjøp", "vedtektsfestet", "bruksrett", "koster", "", "garasjeplassene", "allokert"
                , "utvalgte", "merket", "«prioritet»", "kjøpe", "garasjeplassen", "eventuelt", "tilbys", "kjøpere", "fastsatte", "retningslinjer"
                , "energimerke", "snitt", "forekomme", "bergenes", "før", "overtagelse", "faktura", "sendt", "andelseier", "fjordkraft", "oppvarmingtype"
                , "tappesteder", "kanalvifter", "våtrommene", "skrutypen", "gården", "termostatstyring", "verdt", "merke", "seg", "stavs", "modernisert"
                , "", "brl", "stilren", "", "etasje", "parkeringsplass", "inordning", "tur", "rekreasjonsmuligheter", "koblet", "gjenvinner", "beregning"
                , "foruten", "soveromwic", "leverandør", "hvordan", "boligen?", "utforming", "urbane", "veldefinerte", "gaterom", "torg"
                , "funksjonsblanding", "vektlegges", "kontakt", "koksabukta", "fri", "sikt", "ferdsel", "grønne", "omgivelser", "trygge", "bomiljøer"
                , "forretningssenterets", "grøntkorridor", "hensiktsmessig", "trafikkavvikling", "områdesenteret", "miljøvennlige"
                , "transportløsninger", "intern", "bilbruk", "miljøvennlig", "energiforsyning", "fornybare", "energikilder", "minste", "effektiv"
                , "soilrør", "meste", "rørsystemet", "arbeidene", "borge", "samsvarserklæring", "fortum", "el‑tavle", "el‑skap", "trappeoppgang"
                , "brann", "oppbrukt", "avløpsledninger", "vindusramme", "tappekraner", "levering", "fungerte", "byggeåret", "ctc", "ferro", "l"
                , "", "kjøkkenbenken", "entrégang", "punkt", "utstyret", "enkelt", "tvbredbånd", "tiltalende", "selveier", "løsninger", "inneklima"
                , "innholdsrik", "delikate", "samtidig", "velholdt", "bygningsdeler", "fått", "vedlagt", "datert", "", "‑elektrisk", "‑balansert"
                , "terrassen", "gangentre", "digital", "analog", "medtatt", "lett", "tilgjengelig", "skult", "termostatstyrte", "underjordisk"
                , "biloppstillingsplasser", "laveste", "frie", "høyde", "garasjekjelleren", "", "meter", "garasjehimling", "mineralull", "partier"
                , "garasjeanlegget", "avtrekksventilert", "visse", "lader", "el‑bil", "begrensning", "stikk", "ampere", "sykkelparkering", "terreng"
                , "forbeholder", "retten", "bestemme", "parkeringsplasser", "boder", "bestemmes", "garasjeporten", "motordrift", "fjernbetjening"
                , "bodanlegg", "p‑plass", "nødvendigvis", "betonggulv", "nettingvegger", "hengelås", "årsaker", "full", "ventilasjonsrør", "lignende"
                , "sportsboder", "egner", "gjenstander", "tåler", "temperatur‑", "av", "næringsseksjonen", "", "tilbygget", "endret", "helhetsinntrykk"
                , "eiendom", "bevegelse", "lokalet", "områder", "kunder", "spesielt", "lav", "romhøyde", "terrengforhold", "snø", "fallforhold"
                , "normale", "ledes", "fundamentert", "steinmur", "kledd", "virksomhet", "mur", "tilstøtende", "arealer", "bærende", "veggkonstruksjoner"
                , "liggende", "tømmer", "trepanel", "etasjeskillere", "overliggende", "bestående", "enheter", "vindu", "seksjonen", "midt"
                , "levetidssyklus", "grov", "skjønnsmessig", "befaringsdagen", "fasaden", "adkomstdør", "tofløyet", "utsiden", "raskt"
                , "identifisere", "hovedinngang", "takkonstruksjoner", "saltaksform", "takarker", "taktekking", "metallplater", "snøstoppere"
                , "skippergata", "påvirkes", "nedbør", "xxxxx", "steder", "ild", "besiktigelse", "fleste", "terrengnivå", "tl", "etasjeplan", "lagerrom"
                , "andelen", "si", "elbil‑lading", "ladbare", "hybrider", "parkeringsplassen", "ladepunkt", "dekkes", "seksjonseier"
                , "parkeringsanlegget", "estimert", "månedelige", "", "ovenfor", "inngår", "styrehonorarer", "revisor", "bygning"
                , "heiser", "utomhus", "fellesareale", "tvfibe", "avsetning", "prosjektet", "sameierbrøk", "fastsatt", "seksjoner"
                , "tvfiber", "boligseksjon", "engasjert", "driftsårene", "leilighetsgulv", "bestå", "enstavs", "heltre", "slitesjikt"
                , "eik", "sliteskiktet", "mattlakkert", "børstet", "underliggende", "referer", "oppvarmingssystemet"
                , "keramiske", "kvartal", "varmtvannsystem", "fremlagt", "nilancompact", "inneholder", "filtrert", "forvarmet"
                , "tilføres", "kjøkkenhette", "avtrekkskanal", "ventilasjonsaggregatet", "ført", "etasje", "totalrenovert"
                , "reguleringsplaner", "bakken", "id", "", "http", "webhotelgislineno", "webplan_", "gl_planarkivaspx?planid="
                , "navn", "", "", "hensikten", "reguleringsplanen", "legge", "for", "senter", "fornebuområdets", "visuelle"
                , "funksjonelle", "tyngdepunkt", "handel", "sosiale", "tilbud", "utbygging", "detaljvarehandel", "offentlige"
                , "private", "servicefunksjoner", "kirke", "kultur", "legger", "flotte", "utsikten", "slottsfjell", "sjøen"
                , "innseilingen", "tønsberg", "slipper", "nytes", "sørger", "hygge", "markise", "hagemøbler",  "leiligheteer"
                , "hovedenergikilden", "konsesjonsområde", "planlagt", "akershus", "energibehov", "pkjeller", "stuekjøkken", "prom"
                , "varmegjennvinning", "svalanlegg", "dekker", "pex", "sentralstøvsuger", "fordelerkurser", "", "amp", "hovedkurs"
                , "amp", "elkontroll", "velkommen", "holmenkollveien", "b", "spennende", "attraktivt", "avstand", "friluftsområder"
                , "offkom", "dagligvarehandel", "barnehager", "rikt", "utvalg", "fasiliteter", "høy", "byr", "praktfull", "fjord", "by"
                , "sydvestvendt", "grunnvann", "økonomisk", "besparelse", "vv", "felleskost", "opparbeidet", "trimrom", "kjølerom"
                , "smørebenk", "rullerom", "gjesteparkering", "moderniseringsbehov", "spesielle", "vertikal", "nydalen", "innovative"
                , "løsningene", "kjøling", "svalende", "boligdelen", "konseptet", "klimatisering", "bruke", "passive", "tilføre"
                , "mekanikk", "messige", "tillegg\ttil\tå\t\tredusere\tutgifter\ttil\tdrift\tog", "lar", "møblere", "ovner",
"brannslukking", "varmesystem", "10", "30", "2575", "overbelastningsvern", "11", "81752012", "30052022", "500", "01072022", "197045"
, "2022", "1530", "40", "25100", "1520", "2015", "2016", "2017", "200", "13", "32a", "63", "8", "32", "12", "3", "97m2", "2007", "2lags"
, "2019", "19972010", "3roms", "1991", "14", "2010", "2020", "120l", "2013", "6", "10m²", "45m²", "2006", "4‑roms", "1", "1012", "798‑"
, "2roms", "195", "3‑roms", "138", "25", "900‑", "4191‑", "250", "269", "170", "4mannsboliger", "5", "1997", "55", "2018", "190", "107"
, "3stavs", "800", "600", "80", "m2", "150", 'a', "9", "51", "1995", "25%", "tek17", "172", "2014", "2009", "2007‑2008", "2001"
, "1998‑1999", "135", "2950000", "7990000", "5680", "35", "4", "36", "299", "400000", "1stavs", "2011", "17", "1etasje", "115l"
, "2000", "07032022", "210", "1920", "39", "4etasje", "2002014", "webhotel3gislineno", "webplan_3024", "gl_planarkivaspx?planid=2002014"
, "88", "89", "16", "20amp", "25amp", "111b", "sofamøblement", "dels", "pvcbelagt", "treningssenteret", "hønefoss", "minutter", "unna"
, "bil", "løpe", "bruker", "1015", "oppvarmingsturen", "formen", "ventilasjonsagregatet", "demontert", "fjernet", "igjen", "innhentet"
, "særskilte", "pålegg", "feiervesen", "236m2", "strietapet", "stige", "hems", "lagringsmuligheter", "kneloft", "kontordel", "skråvegg"
, "målbart", "varmekilders", "forkrommende", "ulik", "2004", "2012", "hovedtavle", "nr", "varmer", "eieren", "brannteppe", "tvkrok"
, "kjølige", "vinterdager", "lage", "stemning", "fyre", "utgangen", "balkongdør", "rørkasse", "sisternen", "stakepunkt", "stammen"
, "felleskanal", "fellesopplegg", "oppgang", "fordelingstavle", "isolerglassvinduer", "vesentlig", "trengs", "100%", "klimanøytral"
, "miljøkrav", "beregnes", "smartly", "nøyaktig", "vaskeroms","punkter", "tilstandsrapporten", "tg2", "bom", "takkonstruksjon","huset"
, "bidrar", "eies", "kosan", "vedlikeholder", "tanken", "bestilling", "hos", "innbetaler", "18", "000", "depositum", "siste", "varmeteknikk"
, "hybelleilighet", "hagestuenentreen", "peisestue", "etasjene", "hovedoppvarmingen", "baserer", "radonsikring", "totalvurdering", "målinger"
, "tiltaksgrense", "bemerke", "uetg", "betrakte", "gjennomført", "høyere", "anbefalte", "foreta", "likevel", "utføre", "eventuelle", "måtte"
, "fremkomme", "bakgrunn", "rekkverkshøyde", "forskriftskrav", "100cm", "takvinduer", "tilstand", "umiddelbare", "utskiftninger", "bytte"
, "beslag", "utskifting", "fortløpende", "vurderes", "renner", "nedløp", "hensyn", "funksjonstid", "nådd", "snøfangere", "avviket", "definert"
, "ns3600", "settes", "byggforskserien", "700320", "intervaller", "betongpanner", "intervall", "lang", "60", "vedlikeholdsintervaller", "stein"
, "byttes", "åpninger", "større", "påregnelig", "nærmer", "gjennomføres", "kvalifisert", "elektrofaglig", "person", "skader", "lekkasjer"
, "oppstå", "1984", "anses", "utgått", "antas", "tidsspørsmål", "slutter", "fungere", "halve", "forventbare", "passert", "fortsette"
, "serviceintervallet", "produsenten", "anbefaler", "all", "opprinnelige", "mindre", "svelling", "nedre", "baderomspanel", "gulvbeleggets"
, "tilslutning", "tilfredsstillende", "hjørner", "sveiseskjøter", "intakt", "baderomspaneler", "fuktsikring", "baksiden", "liten", "fare"
, "skade", "bakenforliggende", "påbegynt", "badets", "grunnstamme", "1998", "benyttes", "reservebad", "fks", "renovering", "tettesjikt"
, "registreres", "loftstue","hovedplan", "varmtvannstanken", "brannslokkingsapparat", "hovedoppvarming", "etg", "kjellerstue", "ildsteder"
, "2005", "resterende", "lengre", "lov", "norge", "mange", "tilfeller","bygges","gis", "garanti", "evt", "undersøkes", "nedgravde"
, "graves", "fjernes" , "tømmes", "saneres", "profesjonelt", "firma", "påkoste", "rentbrennende", "ifølge", "innsatser", "tilfredstiller"
, "jøtul", "utleiedel", "usikker", "brensel", "kledning", "etasjeskiller", "membraner", "piper", "feieluker", "plassbygde", "generelle"
, "innerdeler", "spørsmål", "bruksendret", "svar", "ja", "tilbygg", "veranda", "vinterhage", "innebygget", "nb", "egnet", "vinteroppvarming"
, "underetasjen", "etasjen", "gulvfølger", "shiedel", "sargas", "hovedetasjen", "thermia", "sokkel", "verken", "hoveddel", "trappestikk"
, "stua", "nedgravd", "nå", "sanert", "tegl", "pipestokk", "feiervesenet", "oppgraderes", "fortalt", "utestue", "takterrasser", "anneks"
, "kostnadseffektiv", "bakerovn", "verksted", "hobbyrom", "idrettsbane", "volleyballbane", "198", "2½", "sør", "øst", "takterrasse"
, "inntilliggende", "46", "biloppstillingsplass", "lagerarealer", "1954", "inngangsparti", "1978", "1982", "kurant"
, "angitt", "wc‑rom", "sløyfer", "defekt", "gråvann", "skru", "peisinnsats", "kjøkkendel", "panasonic"
, "service", "kjellerstuen", "kjellerboden", "luft", "luft", "300", "2etasje", "kolbe", "brannslukkningsutstyr", "batteri"
, "ventilasjonsaggregat", "kaldtloft", "trappeoppgangen", "rekke", "reduserer", "strømforbruket", "oven", "biopeis", "hovedstuen"
, "nede", "himlingen", "dusjhjørne", "svingbare", "dørbladet", "mellomgang", "innredede", "tomt", "kombipeis", "kjelleren", "indikasjon"
, "enhet", "varmet", "gjøre", "brukes","elektrokolber", "foreligge","flislagte", "supplert", "kobberledninger", "stk"
, "x", "forrige", "produsert", "pumpe", "pumpen", "slitt", "skiftes", "stuene", "stålpipe", "fine", "peiser", "klebersten", "dørterskel"
, "vinylbelegg", "høydeforskjell", "terskel", "tilfredsstille", "forskrift", "minimum", "*", "oppfylt", "skorsteinsløpet", "28072020"
, "romerike", "redningsvesen", "montere", "kontor", "ene", "eneboligen", "plassbygd", "innsats", "omramming", "tilkoplet", "kombibereder"
, "frakoblet", "flisene", "hulrom", "intakte", "solenergi", "utleid", "garasjen", "x2", "metall", "håndslukkerapparat", "luften", "fyres"
, "oppvarmingskilder", "problemer", "skorstein", "feks", "dårlig", "fyringsforbud", "tilsynsrapporten", "redning", "kamerakontrollen"
, "omfattet", "forutsatt", "forårsaker", "avik", "fuger", "teglskorstein", "utslitt", "skorsteinen", "bærer", "preg", "ta", "håndverker"
, "utbedret", "sotlukens", "brennbart", "materiale", "millimeter", "gammelt", "innslag", "tettes", "forsvarlig", "side", "sideinngang"
, "flexit", "oppvarmingskostnad", "klaffeventiler", "flexislange", "stikkledninger", "kommunalt", "nett", "mesteparten", "bunnledning"
, "h", "o", "v", "hørmann", "inngangsdør", "fingeravtrykk", "lås", "sikkerhetslås", "yttertøy", "speildører", "herfra", "spisestuen"
, "inverter", "veksle", "dekorativ", "oppleves", "lyst", "endeveggen", "29", "nedsenket", "spabad", "murte", "levegger", "samlingspunkt"
, "året", "nedenfor", "platting", "22", "vedlikeholdsfrie", "gulvbord", "plenarealer", "holdt", "klassisk", "stil", "hvite", "innredningen"
, "væske", "klargjort", "føre", "konvektor", "varmeovner", "kjelleretasjen", "tidspunkt", "piperehabilitering", "hovedetasje", "møblering"
, "iøynefallende", "lun", "langbord", "invitere", "gjester", "hyggelige", "sammenkomster", "dagslys", "inntrykk", "nærområdet", "naturskjønne"
, "terrasseområde", "utemøbler", "høydedrag", "følelse", "ferdes", "koselig", "samle", "familie", "venner", "sene", "sommerkvelder", "separat"
, "nyoppført", "enebolig", "innredninger", "luksuriøse", "solide", "materialvalg", "helhetlig", "alt", "grunnbelysning", "downlights"
, "ledlamper", "designlamper", "kaldeste", "dagene", "systemer", "stål", "friskluftsventil", "utlufting", "vinduernfor", "sikringstavle"
, "seriemåler", "hovedhus", "takhøyde", "hagen", "microsement", "spiledetaljer", "badekar", "dansk", "modell", "rais", "visio", "svart"
, "ramme", "ytre", "kjellerrom", "dovre", "kloakkledning", "1999", "kloakkrørsystemet", "plastbelegg", "sluker", "rustfritt", "grovkjøkkenet"
, "rørføring", "nibe", "135543", "kw", "borehull", "à", "200m", "takstmannen", "nevnt", "vedrørende", "rapporten", "avvikene", "utbedres"
, "branntilsynet", "inngangspartiet", "hybelen","element", "suppleres",  "19022021", "100cb", "f220", "merknader", "virker"
, "bare", "hentet", "norsk", "gjenvinning", "januar", "forbudt", "fortsatt", "krever", "kommuner", "helt", "fylles", "tillatelse", "kommunen"
, "fjerning", "sanering", "påløpe", "underlag", "metallfolie", "optimalisere", "effekten", "oppvarmingstiden", "redusere", "strømkostnadene"
, "kvalitetslaminat", "fotlister", "tørre", "flisekompaniet", "lekkert", "flisevalg", "linn", "skuffefronter", "klosett", "garnityr", "rørlegger",
"svenner", "https", "rorleggersvennerno", "tekstilfront", "tekstilfronter", "skyvedørsgarderober", "hth", "integrert", "bosch", "siemens",
"induksjonstopp", "spilevegger", "spiler", "himlinger", "trappetrinn", "opptrinn", "sandefjord", "sitteplasser", "lysmalt", "finner", "profilerte", 
"dobbel", "oppvaskkum", "ettgreps", "blandebatteri", "kokesonen", "stekeovn", "avsatt", "kjøleskap", "sittegruppe", "forbindelse", "dobbeltseng",
"skrivebord", "garderobe", "entréen", "yttergang", "daikin", "feiier", "feiing", "u", "drenering", "glassfiber", "husholdninger", "fases", "innen", 
"tanker", "destrueres", "ansvaret", "magasinovn", "kommentar", "lekerom", "andreetasje", "27", "m3", "betjener", "uttak", "gassgrill", "uteareal", 
"vedovni", "varm", "hovedetasjens", "trepanelr", "gulvbelegg", "furugulv", "støpt", "grunnen", "trebjelkelag", "teglsteinpipe", "uinnredet",
"ubehandlet", "betong", "grunnmur", "trinn", "furu", "fyllingsdører", "primært", "stedvise", "vestvegen", "280", "gårdsplass", "belegningsstein", 
"hyggelig", "velkomstterrasse", "inne", "spisestuemøbler", "betongpipe", "carmepumpe", "lukket", "bes", "orientere", "dessuten", "undersiden", 
"elekrisitet", "butikklokale", "75", "fuktskader", "dreneringen", "punktvis", "nederst", "terrenget", "byggverket", "lede", "overvann", "fallet",
"veggen", "tilfelle", "forhindre", "fuktpåkjenninger", "grunnmuren", "trekledning", "påvirkninger", "kledningen", "lengste", "sammenheng", "nevnte",
"tildelt", "tekkingen", "byggforsks", "byggdetaljblader", "takrenner", "plastbelagt", "loftsrom", "utlevd", "punktert", "jevne", "mellomrom",
"teglsteinspiper", "pipens", "kjellere", "måten", "ildstedene", "befaringen", "lekkasjevarsling", "overflateslitasje", "levettid",
"trappeløpet", "løfter", "informasjon", "dokumentert", "eswa", "bjelkelag", "uvisst", "innmat", "brannslukker", "tank", "oppgraderte",
"flater", "laminerte", "skrog", "laminert", "kjøkkenkum", "avkast", "2002", "beleggsoppbrett", "sersvant", "sofaseksjon", 
"tilleggskommentar", "robotklipper", "drivhus", "vannrenseanlegg", "bygge", "svømmebasseng", "terapibadekaret", "tok", "inntak", 
"krypkjeller", "låve", "yttertak", "fyrrom", "nåværende", "tanke", "peiskasset", "sertifisert", "montør", "kjellervinduene", "knuste",
"ruter", "håndtak", "teipet", "platene", "koketopp", "bryter", "bråk", "kjøkkenvifte", "blåser", "boareal", "benytte", "gjesterom",
"tilstøtenderom", "panellering", "inkludere", "slette", "påstøp", "klart", "steuen", "", "alt", "revet", "29102013",
"17122020", "hovedvannledning", "elvifte", "loftsetasje", "belagt", "skap‑", "benkeplass", "utslagsvask", "vaskemakin",
"innfor", "kjellerrommet", "søkt", "bruksendring", "dermed", "kondemnert", "hovedsoverom", "våtromstapet", "slettevegger", 
"utslagskum", "kum", "sprekk", "ildstedet", "komplette", "familieboligen", "fjellstrand", "farge", "laminatgulv", "1peis", "2peis", 
"3vedfyrt", "4ovn", "master", "bedroom", "5ovn", "6varmekabler", "oppe", "7varmekabler", "8varmekabler", "skifergulv", "9varmekabler",
"10varmekabler", "2fløyet", "bioolje", "fyranlegg", "avslått", "varmekolbe", "oppføring", "77", "3000", "glassfibertank", "kjellertrapp",
"vanskelig", "åpne", "karm", "lukkes", "malingflass", "karmer", "tyde", "feil", "vinterstid", "oppsyn", "vurdere", "utnyttet", 
"sengeplassen", "familiekøyeseng", "dobbelt", "strie", "takess", "ihht", "peisoven", "utestikk","lysmalte","elbillader",
"sokkeletasje", "kjelleretasje", "entregulv", "innerste", "minus", "stråle", "reflektorovn", "vhja", "fargevalg", 
"baderomsinnredning", "toppmontert", "gulvmontert", "massasjedyser", "lufteventil", "veldig", "ujevn", "trolla", "414",
"fritidsbolig", "insats", "skilte", "fantastiske", "uteplasser", "hvorav", "særlig", "takterrassen", "nevnes", "sommerdager", 
"massevis", "sol", "smakfullt", "parkettgulv", "tidløse", "luftige", "pene", "w", "1989", "friskluftsventiler",
"ventilasjonsvinduer", "287", "1996", "brannmur", "delen", "1955", "delt", "møbleringsalternativer", "stuedelen", 
"sone", "salongmøbler", "midtpunkt", "spisestuedelen", "langstrakt", "omkring", "tilbygd", "tilbaketrukket", "hellelagt",
"omgjøres", "stilfull",  "bergen", "ovnene", "contura", "uttrykk", "våtromspanel", "oppbrett", "veggskap", "speilskap",
"innfellbare", "herdet", "ganggulv", "unknown", "oppvarmingsmuligheter", "overetasjen", "gruslagt", "møtt", "romslige",
"langsiden", "bord", "sin", "naturlige", "dekke", "måltider", "benkeplatene", "overskapene", "kjøle", "fryseskap", 
"mikrobølgeovn", "helfliset", "mørk", "teglpipe", "oldsteder", "funksjons", "indikert", "valgt", "veggbrennbart",
"cm", "forskrifts", "brannplate", "tiltenkt", "innstallert", "59", "avfall", "boligbebyggelse", "tilkobles",
"igangsettingstillatelse", "bygging", "tilkopling", "vannforsyning", "planen", "godkjennes", "trafo", "agder",
"utbyggingen", "praksis", "regulere", "planforslag", "alternativ", "jord", "planområdet", "planlagte", "veger", 
"vendehammer", "oppfyller", "renovasjonsbil", "lastebil", "oppstilling", "søppeldunker", "skje", "frysedel", 
"møbleringsmuligheter", "1‑", "hentes", "utleie‑enheten", "infrarøde", "infrarød", "strømbesparende", "informasjonsskriv",
"mottatt", "pipen", "sjekket", "fagmann", "etgbad", "fyr", "fyren", "monterte", "visningen", "stålpiper", 
"1970", "meldt", "vestfold", "interkommunale", "brannvesen", "meldepliktig", "innmelding", "påfølgende",  "innunder",
"trappen", "galvanisert", "glassdør", "luft2019", "innedel",  "feieluke", "plate", "dagtank", "brannmurer", "puss", 
"solskjerming", "solfilm", "temperaturen", "foretrekker", "taes", "innrede", "soneinndeling", "bibliotek", "musikksone",
"flygel", "komfortsone", "sørvendt", "manuell", "innefra", "flislag", "mitsubishi", "kaiteki", "slippes","tvstue", "hoveddelen",
"fremført", "400v", "ledning", "hurtigoppvarming", "komfortabelt", "hvis", "husstander", "nylig", "motor", "1½", "1960", "1980",
"blandet", "19802003", "vainstallasjoner", "boligoppvarming", "organiske", "byggematerialer", "tilfarerkonstruksjoner",
"diffusjonsbegrensede", "påforinger", "utførelsen", "risikokonstruksjoner", "forbundet", "helhet", "fremsto", "fuktbelastning", 
"betraktning", "fuktrelaterte", "skjulte", "krypkjellere", "fuktproblematikk", "hovedregelen", "forholdet", "inntrinn",
"stigningsvinkel", "trappeformelen", "=", "620", "±", "20mm", "brattere", "harde", "trenger", "overflatebehandling", "tildekket",
"sider", "avdekke", "sprekkdannelser", "ovennevnte", "dimensjonering", "klamring", "innfesting", "vannmengde", "tapping",
"tilkoblingspunkt", "innført", "fuktsøk", "utsatte", "støpejernsluk", "veggfliser", "forskjellig", "utseende", "treg", "tett", 
"utdatert", "tettefunksjon", "dusjing", "kabinett", "påkrevd", "generell", "basis", "alltid", "omsetning", "lokale", "fører", 
"elementpipen", "teglsteinspipen", "grunnet", "knust", "klar", "2008", "hybel", "kleberstein", "25102021", "17072018", 
"leveransebeskrivelsen", "renoveres", "tetthets", "funksjonsprøvd", "sokkeletasjens", "stod", "stueoppvarming", "håndkleholder", 
"ferdigattest", "plassbygget", "trappenedgang", "heldekkende", "trapperommet", "200l", "viken", "smartstyring", "lysene", 
"strømkabel", "eiendommene", "værslitt", "påbegynnende", "råte", "mest", "hjørnekasse", "belistning", "vannbrett", "faller",
"uheldig", "murer", "setninger", "tilsier", "kosmetiske", "egeninnsats", "ujevne", "plater", "grunnlaget", "flisarbeid", "anntatt",
"eregistrert", "løse", "sanitærutstyr", "løsner", "våtrommet", "tilluftsventilering", "tomteforhold", "fundamenter",
"tegelsteinspipe", "salgsoppgaven", "entreen", "komponenter", "kjellervegg", "sanitærinstallasjoner–", "lansering", 
"antar", "1990", "brystningspanel", "minikjøkken", "benkkjøleskap", "mini", "komfyr+egen", "renovert", "smal", 
"himlingsventil", "spalte", "dørblad", "uft", "kjøpte", "årgang", "innendørs", "hvitmalte", "vel", "tydelig", "ekte",
"gamlebyenhjem", "bevarte", "rosett", "tømmervegger", "tregulv", "generøs", "deles", "blikkfang", "høst", "vinterkvelder", 
"middag", "trekke", "dagligstuen", "salong", "biblioteket", "fjerde", "arbeidsrom", "hva", "hjemmekontor", "historiske", 
"omgivelsene?", "wcrom", "103", "tilførsel", "pollenfilter", "innsugningssiden", "monter", "termostatstyrt", "under‑", "soveromsgangen",
"nobø", "styringspanel", "preget", "trekonstruksjoner", "radonmåling", "grenseverdier", "lettklinker", "ringmur", "kryperom",
"tredører", "2550", "h202", "1986", "h102", "130", "sigdal", "h201", "h101", "vvb", "elanlegget", "dusjvegger", "ventileres",
"glassdører", "dusjsonen", "schiedel","tiden", "forsyne", "produserer", "7500", "kwh", "rør‑i‑rør", "frisk", "tilført",
"aktiv", "trekkes", "lufterør", "avløpet", "el‑anlegg", "skru‑", "kjøkkengulv", "kombinasjon", "grunnbelysningen", "ivaretas",
"spotskinner", "stålforing",  "forsynes", "håndklevarmere", "gjennomføring", "opus", "ignis", "kurs", "vaskeromg", "stålrør",
"brannsikker", "kjøkkeninnredningen", "lovlighet", "godkjente", "plantegninger", "påført", "rombenevnelser", "beskrevet", "2060",
"multielementer", "risikokonstruksjon", "kjent", "elementene", "hatt", "tekking", "3060", "taksering", "norges", "databaser", "estimeres",
"4050", "tgg", "knirk", "sparkling", "sparklingen", "røroppleggene", "fallsikring", "nivåforskjeller", "05", "sikres", "nivå", 
"nivåforskjell", "06", "dusjsone", "slukhøyde", "gulvkonstruksjonen", "sannsynligvis", "medføre", "kategorien", "fukttett", "laget",
"oppkant", "glassfelt", "automatiske", "175", "økt", "vanninntrengning", "årsaken", "manglende", "«drenering»", "rifter",
"oppdriftsventilasjonen", "tørkeskap", "dusjrom", "techem", "ovnsdør", "curve", "100", "asfaltert", "daglig", "verandaen",
"gressplen", "lekestue", "omfattende", "etasjeovn", "strålelampe", "sette", "utsiktsforhold", "eidangerfjorden", "omkringliggende",
"uteplass", "fremsiden", "sofagruppen", "mdf", "veggflater", "hvitmalt", "selvundersøke", "tiltalk", "elkjel", "øvirg"]

In [112]:
Heatingtype = {
"district heating": ["fjernvarmeanlegg","fjernvarme","fjernvarmenettet","fjernvarmeanlegget", "fjernvarmetilknytning", "fjernvarme", "fjernvarmesentral","fjernvarmen"],

 "radiators": [ "radiatorer", "panelovner", "radiator", "panel","panelovn", "panelover", "radiatorovner","radiatorovn", "radiatorene", "stråleovner","varmeovn" ],
 
"floor heating": ["gulvvarme", "varmekabler", "varmekabel", "gulvvarmen", "gullvarme", "gulvarme","varmekabelr","varmkabler"],

"electrical": ["elektriske", "elektrisk", "elekrtrisk", "elektrisitert", "elfyrkjele", "el‑kjele","elkjele","elvarme","elgulvvarme" ],

"heat pump": ["varmepumpe","varmepumpen","varmepumper","varmepumpeanlegg","avkastvarmepumpe","varmepumpesystem","luftvarmepumpe","bergvarmepumpen","varmepumpumpe","varmepumpene"],

"central heating": ["sentralfyring", "sentralvarmeanlegg", "sentralvarme", "sentralanlegg","fellesanlegg", "varmesentral", "varmeanlegget", "sentralfyr", "fyringsanlegg","fyringsanlegget"],

"hot water heating": ["vannbårent","vannboren","vannbårenvarme","luft‑til‑vann","lufttilvann"],

"hot air heating" : ["varmluft","luft‑til‑luft", "lufttilluft", "luft-luft","luftbåren","luft‑luft" ],

"energy saving measures": ["varmefolie", "vindfang", "varmegjenvinning","varmegjenvinner", "energigjenvinning","varmematte","varmekonvektor", "varmeveksler", "jordvarme" ,"isolerglassvinduer","energibrønner", "varmefolien","gjenvinningsvarmepumpe","solcellepanel","varmegjenvinneren","depronfolie", "solcellepaneler"],

"bioenergy": ["vedfyring", "vedovn", "peis", "peisovn","bioenergi","kamin", "klebersteinspeis", "peisen", "kakkelovn","fyrkjele","vedfyrt","biofyringsolje", "bioparafin","biodrivstoff","vedovner","vedovns","peisovner","designpeis","pelletsovn","elementpeis","klebersteinovn","vedoppfyring", "gruepeis", "vedfyringsovn", "vedkamin", "klebersteinsovn","vedovnen"],


"oil and gas": ["gasspeis", "parafinovn", "oljefyring", "gasspeiser","gasstank","gassanlegg", "oljekamin","fossil", "olje", "parafin","parafinfyringsanlegg","oljetanker","oljetank","parafintank","mineralolje","oljefyrte", "oljetanken","gassovn","oljefyr","oljefyringsanlegg","oljefyrt", "parafinbrenner","gassanlegget" ]}



In [113]:
def FindHeating(input):
    heating = []
    if input  != []:
        el = input[0]
        if el == []:
            el = input [1]
        el = " ".join(el)
        el = el.split(" ")
        el = [str.strip(x).replace(".","") for x in el]
        el = [x.replace(",","") for x in el]
        el = [x.replace("/","") for x in el]
        el = [x.replace(")","") for x in el]
        el = [x.replace("(","") for x in el]
        el = [x.replace(r"\\","") for x in el]
        el = [x.replace(r"-","") for x in el]
        el = [x.replace(r"|","") for x in el]
        el = [x.replace(":","") for x in el]
        el = [x.replace('"',"") for x in el]
        el = [x for x in el if x not in garbage_list]
        heating.extend(el)
        
    return heating

def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val in value:
             return key
    return None

In [114]:
## heating
i = 1
folderUrl = "./DNB/JSON/Untreated/Rekkehus/"
withValues = 0
wihtoutValues = 0
garbage_words= []

for file in os.listdir(folderUrl):
    print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        Value = FindHeating(bolig.heating)
        heating_type = []
        for word in Value:
            type = get_key(word,Heatingtype)
            if type != None and type not in heating_type:
                heating_type.append(type)
                continue
            if word not in garbage_words and type == None:
                    garbage_words.append(word)
        print(heating_type)
        i += 1
print(garbage_words)


document number 1 0148767a-a9aa-43b7-71dd-08da511399c3.json
['electrical', 'floor heating']
document number 2 0ac6c21b-3270-47d2-545c-08da107bf601.json
['electrical', 'energy saving measures']
document number 3 0ce8c6fb-2694-4c35-a28c-08da4321c1c3.json
['electrical', 'radiators', 'floor heating', 'heat pump', 'bioenergy']
document number 4 0d0b401a-de44-41ec-3000-08da3fd57391.json
['electrical', 'radiators', 'floor heating', 'heat pump']
document number 5 0fe50b23-5c71-4657-9817-08da3971744c.json
['bioenergy', 'electrical', 'floor heating', 'energy saving measures']
document number 6 107a2a1b-9319-43da-05d3-08da4e3fe05d.json
['bioenergy']
document number 7 10c7fd1b-f5f0-44ae-9903-08da1c8069b2.json
['bioenergy']
document number 8 1a264cf8-32d2-46de-0427-08da55bbf5f2.json
['bioenergy', 'electrical']
document number 9 2408923b-20b0-423d-b0d2-08da4fdacd99.json
['radiators', 'bioenergy']
document number 10 243998cb-4bc3-46f5-a4b5-08da50a2927c.json
['heat pump']
document number 11 24615ada-f